<a href="https://colab.research.google.com/github/DerekLiu35/CS485-project/blob/main/math_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## math utils

In [1]:
def _fix_fracs(string):
    substrs = string.split("\\frac")
    new_str = substrs[0]
    if len(substrs) > 1:
        substrs = substrs[1:]
        for substr in substrs:
            new_str += "\\frac"
            if substr[0] == "{":
                new_str += substr
            else:
                try:
                    assert len(substr) >= 2
                except:
                    return string
                a = substr[0]
                b = substr[1]
                if b != "{":
                    if len(substr) > 2:
                        post_substr = substr[2:]
                        new_str += "{" + a + "}{" + b + "}" + post_substr
                    else:
                        new_str += "{" + a + "}{" + b + "}"
                else:
                    if len(substr) > 2:
                        post_substr = substr[2:]
                        new_str += "{" + a + "}" + b + post_substr
                    else:
                        new_str += "{" + a + "}" + b
    string = new_str
    return string

def _fix_a_slash_b(string):
    if len(string.split("/")) != 2:
        return string
    a = string.split("/")[0]
    b = string.split("/")[1]
    try:
        a = int(a)
        b = int(b)
        assert string == "{}/{}".format(a, b)
        new_string = "\\frac{" + str(a) + "}{" + str(b) + "}"
        return new_string
    except:
        return string

def _remove_right_units(string):
    # "\\text{ " only ever occurs (at least in the val set) when describing units
    if "\\text{ " in string:
        splits = string.split("\\text{ ")
        assert len(splits) == 2
        return splits[0]
    else:
        return string

def _fix_sqrt(string):
    if "\\sqrt" not in string:
        return string
    splits = string.split("\\sqrt")
    new_string = splits[0]
    for split in splits[1:]:
        if split[0] != "{":
            a = split[0]
            new_substr = "\\sqrt{" + a + "}" + split[1:]
        else:
            new_substr = "\\sqrt" + split
        new_string += new_substr
    return new_string

def _strip_string(string):
    # linebreaks
    string = string.replace("\n", "")
    #print(string)

    # remove inverse spaces
    string = string.replace("\\!", "")
    #print(string)

    # replace \\ with \
    string = string.replace("\\\\", "\\")
    #print(string)

    # replace tfrac and dfrac with frac
    string = string.replace("tfrac", "frac")
    string = string.replace("dfrac", "frac")
    #print(string)

    # remove \left and \right
    string = string.replace("\\left", "")
    string = string.replace("\\right", "")
    #print(string)

    # Remove circ (degrees)
    string = string.replace("^{\\circ}", "")
    string = string.replace("^\\circ", "")

    # remove dollar signs
    string = string.replace("\\$", "")

    # remove units (on the right)
    string = _remove_right_units(string)

    # remove percentage
    string = string.replace("\\%", "")
    string = string.replace("\%", "")

    # " 0." equivalent to " ." and "{0." equivalent to "{." Alternatively, add "0" if "." is the start of the string
    string = string.replace(" .", " 0.")
    string = string.replace("{.", "{0.")
    # if empty, return empty string
    if len(string) == 0:
        return string
    if string[0] == ".":
        string = "0" + string

    # to consider: get rid of e.g. "k = " or "q = " at beginning
    if len(string.split("=")) == 2:
        if len(string.split("=")[0]) <= 2:
            string = string.split("=")[1]

    # fix sqrt3 --> sqrt{3}
    string = _fix_sqrt(string)

    # remove spaces
    string = string.replace(" ", "")

    # \frac1b or \frac12 --> \frac{1}{b} and \frac{1}{2}, etc. Even works with \frac1{72} (but not \frac{72}1). Also does a/b --> \\frac{a}{b}
    string = _fix_fracs(string)

    # manually change 0.5 --> \frac{1}{2}
    if string == "0.5":
        string = "\\frac{1}{2}"

    # NOTE: X/Y changed to \frac{X}{Y} in dataset, but in simple cases fix in case the model output is X/Y
    string = _fix_a_slash_b(string)

    return string

def is_equiv(str1, str2, verbose=False):
    if str1 is None and str2 is None:
        print("WARNING: Both None")
        return True
    if str1 is None or str2 is None:
        return False

    try:
        ss1 = _strip_string(str1)
        ss2 = _strip_string(str2)
        if verbose:
            print(ss1, ss2)
        return ss1 == ss2
    except:
        return str1 == str2

In [2]:
import pprint

def last_boxed_only(sample):
    """
    Given a (q,a) sample, filter the answers so that they only contain
    the last \boxed{...} or \fbox{...} element
    """
    q, a = sample
    a = last_boxed_only_string(a)
    if a == None:
        return None
    return (q, a)

def last_boxed_only_string(string):
    idx = string.rfind("\\boxed")
    if idx < 0:
        idx = string.rfind("\\fbox")
        if idx < 0:
            return None

    i = idx
    right_brace_idx = None
    num_left_braces_open = 0
    while i < len(string):
        if string[i] == "{":
            num_left_braces_open += 1
        if string[i] == "}":
            num_left_braces_open -= 1
            if num_left_braces_open == 0:
                right_brace_idx = i
                break
        i += 1

    if right_brace_idx == None:
        retval = None
    else:
        retval = string[idx:right_brace_idx + 1]

    return retval

def only_until_first_boxed_from_tokens(string, tokens):
    idx = string.find("\\boxed")
    if idx < 0:
        idx = string.find("\\fbox")
        if idx < 0:
            return None

    cum_length = 0
    for i, t in enumerate(tokens):
        cum_length += len(t)
        if cum_length >= idx:
            break

    return tokens[:i]



def clean_numbers(sample):
    if not sample:
        return None
    new_sample = list()
    for s in sample:
        new_sample.append(_clean_numbers(s))

    return tuple(new_sample)

def _clean_numbers(string):
    """
    Clean Numbers in the given string

    >>> _clean_numbers(None, "Hello 123")
    'Hello 123'
    >>> _clean_numbers(None, "Hello 1234")
    'Hello 1,234'
    >>> _clean_numbers(None, "Hello 1234324asdasd")
    'Hello 1,234,324asdasd'
    """
    num_prev_digits = 0
    new_string = ""
    for i, c in enumerate(string):
        # isdigit() doesnt work here because of weird unicode chars.
        if c in {'1', '2', '3', '4', '5', '6', '7', '8', '9', '0'}:
            num_prev_digits += 1
        else:
            if num_prev_digits > 3:
                # Some fixing
                string_number = new_string[-num_prev_digits:]
                new_string = new_string[:-num_prev_digits] + "{0:,}".format(int(string_number))
            num_prev_digits = 0
        new_string += c

    if num_prev_digits > 3:
        # Some fixing
        string_number = new_string[-num_prev_digits:]
        new_string = new_string[:-num_prev_digits] + "{0:,}".format(int(string_number))

    return new_string

## Evaluate

In [3]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [5]:
model_name = "NousResearch/llama-2-7b-chat-hf"

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [6]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
!pip install -q dataset

In [8]:
!pip install xformers

In [9]:
from datasets import load_dataset

dataset = load_dataset("hendrycks/competition_math")

In [10]:
problem = dataset['test']['problem'][0]
train_prompt = "Given a mathematics problem, determine the answer. Simplify your answer as much as possible." + "\n" + "Problem: What is $\left(\\frac{7}{8}\\right)^3 \cdot \left(\\frac{7}{8}\\right)^{-3}$?" + "\n" + "Answer: $1$"
train_prompt += "\n" + "###" + "\n" + "Problem: In how many ways can 4 books be selected from a shelf of 6 books if the order in which the books are selected does not matter?" + "\n" +"Answer: $15$"
train_prompt += "\n" +"###" + "\n" + "Problem: Find the distance between the points $(2,1,-4)$ and $(5,8,-3).$" + "\n" + "Answer: $\sqrt{59}$"
train_prompt += "\n" + "###" + "\n" + "Problem: The faces of an octahedral die are labeled with digits $1$ through $8$. What is the probability, expressed as a common fraction, of rolling a sum of $15$ with a pair of such octahedral dice?" + "\n" + "Answer: $\\frac{1}{32}$"
train_prompt += "\n" + "###" + "\n" + "Problem: The first three terms of an arithmetic sequence are 1, 10 and 19, respectively. What is the value of the 21st term?" + "\n" + "Answer: $181$"
train_prompt += "\n" + "###" + "\n" + "Problem: Calculate $6 \\cdot 8\\frac{1}{3}" + "\n" + "Answer: $50$"
train_prompt += "\n" + "###" + "\n" + "Problem: When the binary number $100101110010_2$ is divided by 4, what is the remainder (give your answer in base 10)?" + "\n" + "Answer: $2$"
train_prompt += "\n" + "###" + "\n" + "Problem: How many zeros are at the end of the product 25 $\\times$ 240?" + "\n" + "Answer: $3$" + "\n" + "###"

test_question = "\n" + problem + "\n" + "Answer: $"
prompt = train_prompt + test_question

num_tokens = 20
max_length = len(tokenizer(prompt)['input_ids']) + num_tokens
pipe = pipeline("text-generation", model, tokenizer=tokenizer, max_length=max_length)
tokens = pipe(prompt)[0]['generated_text']

    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


In [11]:
startindex = len(prompt)
endindex = startindex + 1
for token in tokens[startindex + 1:]:
    if token == "$" or token == "###" or token == "\n":
        break
    else:
        endindex += 1
final_answer = tokens[startindex:endindex]
final_answer

'2'

In [12]:
from pprint import pprint
pprint('Given a mathematics problem, determine the answer. Simplify your answer as much as possible.\nProblem: What is $\\left(\\frac{7}{8}\\right)^3 \\cdot \\left(\\frac{7}{8}\\right)^{-3}$?\nAnswer: $1$\n###\nProblem: In how many ways can 4 books be selected from a shelf of 6 books if the order in which the books are selected does not matter?\nAnswer: $15$\n###\nProblem: Find the distance between the points $(2,1,-4)$ and $(5,8,-3).$\nAnswer: $\\sqrt{59}$\n###\nProblem: The faces of an octahedral die are labeled with digits $1$ through $8$. What is the probability, expressed as a common fraction, of rolling a sum of $15$ with a pair of such octahedral dice?\nAnswer: $\\frac{1}{32}$\n###\nProblem: The first three terms of an arithmetic sequence are 1, 10 and 19, respectively. What is the value of the 21st term?\nAnswer: $181$\n###\nProblem: Calculate $6 \\cdot 8\\frac{1}{3}\nAnswer: $50$\n###\nProblem: When the binary number $100101110010_2$ is divided by 4, what is the remainder (give your answer in base 10)?\nAnswer: $2$\n###\nProblem: How many zeros are at the end of the product 25 $\\times$ 240?\nAnswer: $3$\n###\nHow many vertical asymptotes does the graph of $y=\\frac{2}{x^2+x-6}$ have?\nAnswer: $2$\n###\nWhat is the value of $x$ that satisfies the equation $x^2-4x-3=0$?\nAnswer: $x=-\\frac{3\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $a$ that satisfies the equation $a^2-4a-3=0$?\nAnswer: $a=-\\frac{3\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $x$ that satisfies the equation $x^2-4x-1=0$?\nAnswer: $x=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $a$ that satisfies the equation $a^2-4a-1=0$?\nAnswer: $a=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $x$ that satisfies the equation $x^3-2x^2-3x-1=0$?\nAnswer: $x=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $a$ that satisfies the equation $a^3-2a^2-3a-1=0$?\nAnswer: $a=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $x$ that satisfies the equation $x^4-4x^3-6x^2-8x+10=0$?\nAnswer: $x=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $a$ that satisfies the equation $a^4-4a^3-6a^2-8a+10=0$?\nAnswer: $a=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $x$ that satisfies the equation $x^5-5x^4-10x^3-15x^2-20x+25=0$?\nAnswer: $x=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $a$ that satisfies the equation $a^5-5a^4-10a^3-15a^2-20a+25=0$?\nAnswer: $a=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $x$ that satisfies the equation $x^6-6x^5-15x^4-20x^3-25x^2-30x+35=0$?\nAnswer: $x=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $a$ that satisfies the equation $a^6-6a^5-15a^4-20a^3-25a^2-30a+35=0$?\nAnswer: $a=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $x$ that satisfies the equation $x^7-7x^6-14x^5-21x^4-28x^3-35x^2-42x+49=0$?\nAnswer: $x=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $a$ that satisfies the equation $a^7-7a^6-14a^5-21a^4-28a^3-35a^2-42a+49=0$?\nAnswer: $a=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $x$ that satisfies the equation $x^8-8x^7-16x^6-25x^5-35x^4-49x^3-64x^2-80x+99=0$?\nAnswer: $x=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $a$ that satisfies the equation $a^8-8a^7-16a^6-25a^5-35a^4-49a^3-64a^2-80a+99=0$?\nAnswer: $a=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $x$ that satisfies the equation $x^9-9x^8-18x^7-27x^6-36x^5-47x^4-63x^3-84x^2-100x+125=0$?\nAnswer: $x=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $a$ that satisfies the equation $a^9-9a^8-18a^7-27a^6-36a^5-47a^4-63a^3-84a^2-100a+125=0$?\nAnswer: $a=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $x$ that satisfies the equation $x^{10}-10x^9-18x^8-27x^7-36x^6-47x^5-63x^4-84x^3-100x^2-125x+144=0$?\nAnswer: $x=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $a$ that satisfies the equation $a^{10}-10a^9-18a^8-27a^7-36a^6-47a^5-63a^4-84a^3-100a^2-125a+144=0$?\nAnswer: $a=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $x$ that satisfies the equation $x^{11}-11x^{10}-18x^9-27x^8-36x^7-47x^6-63x^5-84x^4-100x^3-125x^2-144x+169=0$?\nAnswer: $x=-\\frac{1\\pm\\sqrt{5}}{2}$\n###\nWhat is the value of $a$ that satisfies the equation $a^{11}-11a^{10}-18a^9-27a^8-36a^7-47a^6-63a^5-84a^4-100a^3-125a^2-144a+169=0$?\n')

('Given a mathematics problem, determine the answer. Simplify your answer as '
 'much as possible.\n'
 'Problem: What is $\\left(\\frac{7}{8}\\right)^3 \\cdot '
 '\\left(\\frac{7}{8}\\right)^{-3}$?\n'
 'Answer: $1$\n'
 '###\n'
 'Problem: In how many ways can 4 books be selected from a shelf of 6 books if '
 'the order in which the books are selected does not matter?\n'
 'Answer: $15$\n'
 '###\n'
 'Problem: Find the distance between the points $(2,1,-4)$ and $(5,8,-3).$\n'
 'Answer: $\\sqrt{59}$\n'
 '###\n'
 'Problem: The faces of an octahedral die are labeled with digits $1$ through '
 '$8$. What is the probability, expressed as a common fraction, of rolling a '
 'sum of $15$ with a pair of such octahedral dice?\n'
 'Answer: $\\frac{1}{32}$\n'
 '###\n'
 'Problem: The first three terms of an arithmetic sequence are 1, 10 and 19, '
 'respectively. What is the value of the 21st term?\n'
 'Answer: $181$\n'
 '###\n'
 'Problem: Calculate $6 \\cdot 8\\frac{1}{3}\n'
 'Answer: $50$\n'
 '###\n'


In [13]:
import os
import numpy as np
import operator
import json
from tqdm import tqdm

def call_engine(train_prompt, problem, engine="davinci"):
    '''
    Given a problem, returns the most likely answer determined by the GPT engine
    '''
    test_question = "\n" + problem + "\n" + "Answer: $"
    prompt = train_prompt + test_question
    # print(len(prompt))
    num_tokens = 20
    # c = openai.Completion.create(
    #                     engine=engine,
    #                     prompt=prompt,
    #                     max_tokens=num_tokens,
    #                     logprobs=100,
    #                     temperature=0,
    #                     echo=True
    #                 )
    # tokens = c["choices"][0]["logprobs"]["tokens"]
    max_length = len(tokenizer(prompt)['input_ids']) + num_tokens
    pipe = pipeline("text-generation", model, tokenizer=tokenizer, max_length=max_length)
    tokens = pipe(prompt)[0]['generated_text'] # This is pretty hacky and probably very slow
    # startindex = -1 * num_tokens
    # endindex = -1 * num_tokens + 1
    startindex = len(prompt)
    endindex = startindex + 1
    for token in tokens[startindex + 1:]:
        if token == "$" or token == "###" or token == "\n":
            break
        else:
            endindex += 1
    # final_answer = ""
    # for i in range(startindex, endindex):
    #     all_answers = c["choices"][0]["logprobs"]["top_logprobs"][i]
    #     best_answer = max(all_answers.items(), key=operator.itemgetter(1))[0]
    #     final_answer += best_answer
    final_answer = tokens[startindex:endindex]
    return final_answer

def remove_boxed(s):
    left = "\\boxed{"
    try:
        assert s[:len(left)] == left
        assert s[-1] == "}"
        return s[len(left):-1]
    except:
        return None

train_prompt = "Given a mathematics problem, determine the answer. Simplify your answer as much as possible." + "\n" + "Problem: What is $\left(\\frac{7}{8}\\right)^3 \cdot \left(\\frac{7}{8}\\right)^{-3}$?" + "\n" + "Answer: $1$"
train_prompt += "\n" + "###" + "\n" + "Problem: In how many ways can 4 books be selected from a shelf of 6 books if the order in which the books are selected does not matter?" + "\n" +"Answer: $15$"
train_prompt += "\n" +"###" + "\n" + "Problem: Find the distance between the points $(2,1,-4)$ and $(5,8,-3).$" + "\n" + "Answer: $\sqrt{59}$"
train_prompt += "\n" + "###" + "\n" + "Problem: The faces of an octahedral die are labeled with digits $1$ through $8$. What is the probability, expressed as a common fraction, of rolling a sum of $15$ with a pair of such octahedral dice?" + "\n" + "Answer: $\\frac{1}{32}$"
train_prompt += "\n" + "###" + "\n" + "Problem: The first three terms of an arithmetic sequence are 1, 10 and 19, respectively. What is the value of the 21st term?" + "\n" + "Answer: $181$"
train_prompt += "\n" + "###" + "\n" + "Problem: Calculate $6 \\cdot 8\\frac{1}{3}" + "\n" + "Answer: $50$"
train_prompt += "\n" + "###" + "\n" + "Problem: When the binary number $100101110010_2$ is divided by 4, what is the remainder (give your answer in base 10)?" + "\n" + "Answer: $2$"
train_prompt += "\n" + "###" + "\n" + "Problem: How many zeros are at the end of the product 25 $\\times$ 240?" + "\n" + "Answer: $3$" + "\n" + "###"

rootdir = "/content/MATH/test"


def run(engine="davinci", max=-1):
    outputs = []
    answers = []
    types = []
    levels = []

    fnames_list = []

    cors = {}
    subject_cors = {}
    level_cors = {}
    correct = 0
    total = 0
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            fnames_list.append(os.path.join(subdir, file))
            with open(os.path.join(subdir, file), 'r') as fp:
                try:
                    problem_data = json.load(fp)
                except Exception as e:
                    print(f"Error loading JSON from {file}", e)
                    raise e
                prob_level = problem_data["level"]
                prob_type = problem_data["type"]
                try:
                    prob_level = int(prob_level.split("Level ")[1])
                except:
                    prob_level = None
                model_output = call_engine(train_prompt, problem_data["problem"], engine=engine)
                answer = remove_boxed(last_boxed_only_string(problem_data["solution"]))

                levels.append(prob_level)
                types.append(prob_type)
                outputs.append(model_output)
                answers.append(answer)

                print("Model output:")
                print(model_output)
                print("Correct answer:")
                print(answer)
                print("--------------------------------------------")

                try:
                    equiv = is_equiv(model_output, answer)
                except:
                    equiv = False
                if (prob_level, prob_type) in cors:
                    cors[(prob_level, prob_type)].append(equiv)
                else:
                    cors[(prob_level, prob_type)] = [equiv]
                if prob_level in level_cors:
                    level_cors[prob_level].append(equiv)
                else:
                    if prob_level is not None:
                        level_cors[prob_level] = [equiv]
                if prob_type in subject_cors:
                    subject_cors[prob_type].append(equiv)
                else:
                    if prob_type is not None:
                        subject_cors[prob_type] = [equiv]
                if equiv:
                    correct += 1
                total += 1

                print(str(correct) + "/" + str(total))

            if max > 0 and total > max:
                break
        if max > 0 and total > max:
            break

    with open("outputs_answers_gpt3_{}.txt".format(engine), "w+") as f:
        for k, (output, answer, prob_type, prob_level, fname) in enumerate(zip(outputs, answers, types, levels, fnames_list)):
            f.write("{} TYPE: {} | LEVEL: {} | OUTPUT: {} | ANSWER: {} | FNAME: {}\n".format(k, prob_type, prob_level, output, answer, fname))

        f.write("#####################\n")
        # also get accuracies for each
        for subject in ['Prealgebra', 'Algebra', 'Number Theory', 'Counting & Probability', 'Geometry', 'Intermediate Algebra', 'Precalculus']:
            for level in range(1, 6):
                key = (level, subject)
                if key not in cors.keys():
                    print("Skipping", key)
                    continue
                cors_list = cors[key]
                print("{} Level {} Accuracy = {}/{} = {:.3f}".format(subject, level, np.sum(cors_list), len(cors_list), np.mean(cors_list)))
                f.write("{} Level {} Accuracy = {}/{} = {:.3f}\n".format(subject, level, np.sum(cors_list), len(cors_list), np.mean(cors_list)))
        print("#####################")
        f.write("#####################\n")
        for level in sorted(level_cors):
            if level not in level_cors.keys():
                print("Skipping", level)
                continue
            cors_list = level_cors[level]
            print("Level {} Accuracy = {}/{} = {:.3f}".format(level, np.sum(cors_list), len(cors_list), np.mean(cors_list)))
            f.write("Level {} Accuracy = {}/{} = {:.3f}\n".format(level, np.sum(cors_list), len(cors_list), np.mean(cors_list)))
        print("#####################")
        f.write("#####################\n")
        for subject in ['Prealgebra', 'Algebra', 'Number Theory', 'Counting & Probability', 'Geometry', 'Intermediate Algebra', 'Precalculus']:
            if subject not in subject_cors.keys():
                print("Skipping", subject)
                continue
            cors_list = subject_cors[subject]
            print("{} Accuracy = {}/{} = {:.3f}".format(subject, np.sum(cors_list), len(cors_list), np.mean(cors_list)))
            f.write("{} Accuracy = {}/{} = {:.3f}\n".format(subject, np.sum(cors_list), len(cors_list), np.mean(cors_list)))
        print("#####################")
        f.write("#####################\n")
        print("Overall Accuracy = {}/{} = {:.3f}".format(correct, total, correct/total))
        f.write("Overall Accuracy = {}/{} = {:.3f}\n".format(correct, total, correct/total))

In [14]:
!wget https://people.eecs.berkeley.edu/~hendrycks/MATH.tar
!tar -xvf /content/MATH.tar

Streaming output truncated to the last 5000 lines.
MATH/test/algebra/2130.json
MATH/test/algebra/1659.json
MATH/test/algebra/1209.json
MATH/test/algebra/816.json
MATH/test/algebra/1570.json
MATH/test/algebra/1065.json
MATH/test/algebra/400.json
MATH/test/algebra/2649.json
MATH/test/algebra/2167.json
MATH/test/algebra/791.json
MATH/test/algebra/2022.json
MATH/test/algebra/1527.json
MATH/test/algebra/512.json
MATH/test/algebra/142.json
MATH/test/algebra/841.json
MATH/test/algebra/904.json
MATH/test/algebra/1462.json
MATH/test/algebra/1032.json
MATH/test/algebra/338.json
MATH/test/algebra/1248.json
MATH/test/algebra/1862.json
MATH/test/algebra/441.json
MATH/test/algebra/2576.json
MATH/test/algebra/2126.json
MATH/test/algebra/2433.json
MATH/test/algebra/945.json
MATH/test/algebra/416.json
MATH/test/algebra/1423.json
MATH/test/algebra/1264.json
MATH/test/algebra/251.json
MATH/test/algebra/1634.json
MATH/test/algebra/314.json
MATH/test/algebra/744.json
MATH/test/algebra/2274.json
MATH/test/a

In [15]:
for engine in ["ada"]:
  run(engine, max=5)

Model output:
49
Correct answer:
144
--------------------------------------------
0/1
Model output:
2 - \sqrt{3} + 2i
Correct answer:
-8 + 2i
--------------------------------------------
0/2
Model output:
 (4,0)
Correct answer:
(4,\pi)
--------------------------------------------
0/3
Model output:
16
Correct answer:
97
--------------------------------------------
0/4
Model output:
2\sqrt{2}
Correct answer:
78 \pi
--------------------------------------------
0/5
Model output:
3.576
Correct answer:
-2
--------------------------------------------
0/6
Skipping (1, 'Prealgebra')
Skipping (2, 'Prealgebra')
Skipping (3, 'Prealgebra')
Skipping (4, 'Prealgebra')
Skipping (5, 'Prealgebra')
Skipping (1, 'Algebra')
Skipping (2, 'Algebra')
Skipping (3, 'Algebra')
Skipping (4, 'Algebra')
Skipping (5, 'Algebra')
Skipping (1, 'Number Theory')
Skipping (2, 'Number Theory')
Skipping (3, 'Number Theory')
Skipping (4, 'Number Theory')
Skipping (5, 'Number Theory')
Skipping (1, 'Counting & Probability')
Sk

In [40]:
from transformers.pipelines.pt_utils import KeyDataset
from datasets import load_dataset
from tqdm import tqdm

pipe = pipeline("text-generation", model, tokenizer=tokenizer, max_length=1000)
dataset = load_dataset("hendrycks/competition_math")

In [41]:
dataset["test"][:5]["problem"]
maxS = 16
total = 0
for out in tqdm(pipe(dataset["test"][:maxS]["problem"], batch_size=maxS)):
    print(out)

100%|██████████| 16/16 [00:00<00:00, 4846.81it/s]

[{'generated_text': "How many vertical asymptotes does the graph of $y=\\frac{2}{x^2+x-6}$ have? Unterscheidung between $y$ and $y'$.\n\nI'm trying to solve this problem, but I'm having trouble understanding how to find the vertical asymptotes of the graph of $y = \\frac{2}{x^2 + x - 6}$. Can someone please help me?\n\nThe problem is asking me to find the number of vertical asymptotes of the graph of $y = \\frac{2}{x^2 + x - 6}$. I know that the vertical asymptotes are the points where the denominator of the fraction goes to infinity, but I'm not sure how to find these points.\n\nI've tried using the following steps:\n\n1. Factor the denominator of the fraction: $x^2 + x - 6 = (x + 3)(x - 2)$\n2. Set the denominator equal to zero: $x + 3 = 0 or x - 2 = 0$\n3. Solve for $x$: $x = -3 or x = 2$\n\nBut I'm not sure if this is the correct way to find the vertical asymptotes. Can someone please help me understand how to find the vertical asymptotes of this graph?\n\nAlso, I'm not sure how to

In [39]:
max(len(p) for p in dataset["test"]["problem"])

2828

### gsm8k


In [45]:
gsm8k = load_dataset("gsm8k", "main")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [43]:
for problem in dataset['test'][:5]['problem']:
  print(problem)

How many vertical asymptotes does the graph of $y=\frac{2}{x^2+x-6}$ have?
What is the positive difference between $120\%$ of 30 and $130\%$ of 20?
Find $x$ such that $\lceil x \rceil + x = \dfrac{23}{7}$. Express $x$ as a common fraction.
Evaluate $i^5+i^{-25}+i^{45}$.
If $2^8=4^x$, what is the value of $x$?


In [57]:
# for q, a in zip(gsm8k['test'][:5]['question'], gsm8k['test'][:5]['answer']):

In [58]:
gsm8k['test']['answer']

['Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18',
 'It takes 2/2=<<2/2=1>>1 bolt of white fiber\nSo the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric\n#### 3',
 'The cost of the house and repairs came out to 80,000+50,000=$<<80000+50000=130000>>130,000\nHe increased the value of the house by 80,000*1.5=<<80000*1.5=120000>>120,000\nSo the new value of the house is 120,000+80,000=$<<120000+80000=200000>>200,000\nSo he made a profit of 200,000-130,000=$<<200000-130000=70000>>70,000\n#### 70000',
 'He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*60=<<9*60=540>>540 meters\n#### 540',
 'If each chicken eats 3 cups of feed per day, then for 20 chickens they would need 3*20=<<3*20=60>>60 cups of feed per day.\nIf she feeds the flock 15 cups of feed in the morning, and 25 cups in the afternoon, then the final meal would require 60-15-25=<<60-15-25=20>>20 cups of chicken feed.\n#### 20',
 'The discount

In [69]:
problem = gsm8k['test']['question'][2]
test_question = "\n" + problem + "\n" + "Answer: $"
prompt = train_prompt + test_question
num_tokens = 20
max_length = len(tokenizer(prompt)['input_ids']) + num_tokens
pipe = pipeline("text-generation", model, tokenizer=tokenizer, max_length=max_length)
tokens = pipe(prompt)[0]['generated_text'] # This is pretty hacky and probably very slow
tokens

'Given a mathematics problem, determine the answer. Simplify your answer as much as possible.\nQuestion: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today? Answer: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6.\nAnswer: $6$\n###\nQuestion: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot? Answer: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5.\nAnswer: $5$\n###\nQuestion: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total? Answer: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39.\nAnswer: $39$\n###\nQuestion: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. Ho

In [67]:
def call_engine(train_prompt, problem, engine="davinci"):
    '''
    Given a problem, returns the most likely answer determined by the GPT engine
    '''
    test_question = "\n" + problem + "\n" + "Answer: $"
    prompt = train_prompt + test_question
    num_tokens = 20
    max_length = len(tokenizer(prompt)['input_ids']) + num_tokens
    pipe = pipeline("text-generation", model, tokenizer=tokenizer, max_length=max_length)
    tokens = pipe(prompt)[0]['generated_text'] # This is pretty hacky and probably very slow
    startindex = len(prompt)
    endindex = startindex + 1
    for token in tokens[startindex + 1:]:
        if token == "$" or token == "###" or token == "\n":
            break
        else:
            endindex += 1
    final_answer = tokens[startindex:endindex]
    return final_answer

def remove_hashtag(s):
    try:
      split_string = s.split('####')
      answer = split_string[-1].strip()
      return answer
    except:
      return None

train_prompt = "Given a mathematics problem, determine the answer. Simplify your answer as much as possible." + "\n" + "Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today? Answer: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6." + "\n" + "Answer: $6$"
train_prompt += "\n" + "###" + "\n" + "Question: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot? Answer: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5." + "\n" + "Answer: $5$"
train_prompt += "\n" + "###" + "\n" + "Question: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total? Answer: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39." + "\n" + "Answer: $39$"
train_prompt += "\n" + "###" + "\n" + "Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny? Answer: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8." + "\n" + "Answer: $8$"
train_prompt += "\n" + "###" + "\n" + "Question: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now? Answer: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9." + "\n" + "Answer: $9$"
train_prompt += "\n" + "###" + "\n" + "Question: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room? Answer: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29" + "\n" "Answer: $29$"  + "\n" + "###"

# rootdir = "/content/MATH/test"

def run_eval(engine="davinci", max=-1):
  outputs = []
  answers = []
  types = []
  levels = []

  fnames_list = []

  cors = {}
  subject_cors = {}
  level_cors = {}
  correct = 0
  total = 0

  for q, a in zip(gsm8k['test']['question'], gsm8k['test']['answer']): # add tqdm progress bar. it already prints stuff tho, might remove that idk
    model_output = call_engine(train_prompt, q, engine=engine)
    answer = remove_hashtag(a)

    outputs.append(model_output)
    answers.append(answer)

    print("Model output:")
    print(model_output)
    print("Correct answer:")
    print(answer)
    print("--------------------------------------------")

    try:
        equiv = is_equiv(model_output, answer)
    except:
        equiv = False
    if equiv:
        correct += 1
    total += 1

    print(str(correct) + "/" + str(total))

    if max > 0 and total > max:
        break

In [68]:
run_eval(max=2)

Model output:
2
Correct answer:
18
--------------------------------------------
0/1
Model output:
3
Correct answer:
3
--------------------------------------------
1/2
Model output:
80,000 \* 150% = 
Correct answer:
70000
--------------------------------------------
1/3


### with huggingface transformers

In [ ]:
import io
import logging
import math
import os
import pprint
import sys
import json
import time
import transformers
import numpy as np

from tqdm import tqdm

import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.multiprocessing as mp

from torch.nn.parallel import DistributedDataParallel as DDP

# from dataset.MATH import MATHDataset
# from dataset.khan_academy import KhanAcademyMathDataset
# from dataset.util import clean_numbers, last_boxed_only, last_boxed_only_string
# from math_equivalence import is_equiv

def get_level_type(fname):
    """
    Somewhat inefficient, but much easier than changing dataloader and probably fine for evaluation
    """
    with open(fname, 'r') as fp:
        try:
            problem_data = json.load(fp)
        except Exception as e:
            print(f"Error loading JSON from {fname}", e)
            raise e
    level, prob_type = problem_data['level'], problem_data['type']
    try:
        level = int(level.split("Level ")[1])
    except:
        level = None
    return level, prob_type

def remove_boxed(s):
    left = "\\boxed{"
    try:
        assert s[:len(left)] == left
        assert s[-1] == "}"
        return s[len(left):-1]
    except:
        return None


def dict_to_gpu(d, device_id=None):
    new_dict = dict()
    for key, value in d.items():
        # Only move to GPU is cuda() is a function
        if 'cuda' in dir(value):
            new_dict[key] = value.cuda(device_id)
        else:
            new_dict[key] = value
    return new_dict


def get_real_sol_idxs(tokens_sol, tokenizer):
    """
    Return the start and stop indexes (inclusive) for everything inside \\boxed{...}
    """
    left_idx, right_idx = None, None
    for i in range(tokens_sol.shape[1]):
        if i < 3:
            continue

        if tokens_sol[0, i].item() and \
            tokens_sol[0, i-1].item() == 276 and \
            tokens_sol[0, i-2].item() == 3524:
            # at index i, we have the { of \\boxed{
            left_idx = i + 1 # Don't include the {

        if tokens_sol[0, i].item() == 50256:
            right_idx = i-2 # don't include the one token before the current one as well (usually the } from \boxed{})

    # Will error if either is not found, which we dont expect
    return left_idx, right_idx


def run_eval(args):

    # argsdict = vars(args)
    # print(pprint.pformat(argsdict))

    # if args.tokenizer_merges_file is not None:
    #     tokenizer = transformers.GPT2Tokenizer.from_pretrained(args.arch, merges_file=args.tokenizer_merges_file)
    # else:
    #     tokenizer = transformers.GPT2Tokenizer.from_pretrained(args.arch)
    tokenizer = tokenizer

    # eval_data = get_dataset(args)
    # for inner_dset in eval_data.datasets:
    #     inner_dset.tokenizer = tokenizer

    # dataloader = torch.utils.data.DataLoader(
    #     eval_data,
    #     batch_size=1,
    #     num_workers=0,
    #     pin_memory=True,
    # )

    """
    with torch.no_grad():
        correct = 0
        total = 0
        for i, batch in enumerate(tqdm(dataloader)):
            batch = dict_to_gpu(batch, device_id=0)
            print(batch['fnames'])
            print(batch['input_ids'])
            quit()
    """

    # Set up model
    if args.load is None:
        model = transformers.GPT2LMHeadModel.from_pretrained(args.arch)
    else:
        print(f"Loading model from {args.load}")
        model = transformers.GPT2LMHeadModel.from_pretrained(args.load)
        print(f"Successfully loaded model from {args.load}")

    # model = model.eval()
    # model = model.cuda()
    model.eval()
    model.cuda()

    loss_moving_average = 0

    outputs = []
    answers = []
    types = []
    levels = []
    fnames_list = []

    cors = {}
    subject_cors = {}
    level_cors = {}

    with torch.no_grad():
        correct = 0
        total = 0
        skipped = 0
        mean_max_probs_correct = []
        mean_max_probs_wrong   = []
        # for i, batch in enumerate(tqdm(dataloader)):

            if torch.sum(batch['input_ids']) == 0:
                skipped += 1
                print("SKIPPING", batch['fnames'][0])
                continue

            fnames = batch['fnames'][0]
            assert len(fnames) == 1
            fnames_list.append(fnames[0])
            prob_level, prob_type = get_level_type(fnames[0])
            batch = dict_to_gpu(batch, device_id=0)

            output_ids = model.generate(
                batch['input_ids'],
                num_beams=args.num_beams,
                early_stopping=True,
                temperature=1.0,
                max_length=384 if args.arch == 'gpt2-xl' else 1024
            )

            # logits = model(output_ids).logits
            # probs = F.softmax(logits, dim=2) # torch.Size([1, L, 50257])
            # max_probs, max_tokens = probs.max(2) # torch.Size([1, L]), torch.Size([1, L])

            # num_tokens_for_question = batch['input_ids'].shape[1]
            # probs_sol = max_probs[:, num_tokens_for_question-1:]
            # tokens_sol = max_tokens[:, num_tokens_for_question-1:]

            # real_sol_start_idx, real_sol_stop_idx = get_real_sol_idxs(tokens_sol, tokenizer)
            # if real_sol_start_idx is None or real_sol_stop_idx is None:
            #     skipped += 1
            #     print("BAD ANSWER, SKIPPING", batch['fnames'][0])
            #     continue
            # probs_sol = probs_sol[:, real_sol_start_idx:real_sol_stop_idx + 1]
            # mean_probs_sol = torch.mean(probs_sol).item()
            mean_probs_sol = 0

            output_tokens = get_model_output(batch['input_ids'][0], output_ids[0], tokenizer)

            # Print this iteration
            output_str = tokenizer.decode(output_tokens)
            output_full = output_str
            output_str = last_boxed_only_string(output_str)

            if args.math_mode == "eval_peeking":
                answer_str = last_boxed_only_string(tokenizer.decode(batch['labels'][0]))
            else:
                answer_str = tokenizer.decode(batch['labels'][0])

            output, answer = remove_boxed(output_str), remove_boxed(answer_str)

            print("Problem String:")
            print(tokenizer.decode(batch['input_ids'][0]) + "\n")
            print("Model output:")
            print(output_full)
            print(output)
            print("Correct answer:")
            print(answer)
            print("fname")
            print(fnames)
            print("--------------------------------------------")

            # scratchwork_fname = "___".join(fnames[0].split("/")[-2:])
            # with open(f"scratchwork_Temp2e-1_{args.arch}/{scratchwork_fname}.txt", 'w') as f:
            #     f.write("Problem String:" + "\n")
            #     f.write(tokenizer.decode(batch['input_ids'][0]) + "\n")
            #     f.write("Model output:" + "\n")
            #     f.write(output_full + "\n")
            #     f.write(str(output) + "\n")
            #     f.write("Correct answer:" + "\n")
            #     f.write(answer + "\n")
            #     f.write("--------------------------------------------" + "\n")

            outputs.append(output)
            answers.append(answer)
            types.append(prob_type)
            levels.append(prob_level)

            equiv = is_equiv(output, answer)
            if (prob_level, prob_type) in cors:
                cors[(prob_level, prob_type)].append(equiv)
            else:
                cors[(prob_level, prob_type)] = [equiv]

            if prob_level in level_cors:
                level_cors[prob_level].append(equiv)
            else:
                if prob_level is not None:
                    level_cors[prob_level] = [equiv]

            if prob_type in subject_cors:
                subject_cors[prob_type].append(equiv)
            else:
                if prob_type is not None:
                    subject_cors[prob_type] = [equiv]

            if equiv:
                correct += 1
                mean_max_probs_correct.append(mean_probs_sol)
            else:
                mean_max_probs_wrong.append(mean_probs_sol)

            # print("CORRECT", mean_max_probs_correct)
            # print("WRONG", mean_max_probs_wrong)

            total += 1

    subjects = ['Prealgebra', 'Algebra', 'Number Theory', 'Counting & Probability', 'Geometry', 'Intermediate Algebra', 'Precalculus']

    print(f"Average of mean_max_probs_correct = {sum(mean_max_probs_correct)}/{len(mean_max_probs_correct)} = ", sum(mean_max_probs_correct)/len(mean_max_probs_correct))
    print(f"Average of mean_max_probs_wrong   = {sum(mean_max_probs_wrong)}/{len(mean_max_probs_wrong)} = ", sum(mean_max_probs_wrong)/len(mean_max_probs_wrong))

    # now save outputs and answers
    with open(f"outputs_answers_Temp2e-1_{args.arch}.txt", "w+") as f:
        for k, (output, answer, prob_type, prob_level, fname) in enumerate(zip(outputs, answers, types, levels, fnames_list)):
            f.write("{} TYPE: {} | LEVEL: {} | OUTPUT: {} | ANSWER: {} | FNAME: {}\n".format(k, prob_type, prob_level, output, answer, fname))

        # print(cors)
        for prob_type in subjects:
            for prob_level in [1, 2, 3, 4, 5]:
                if (prob_level, prob_type) in cors:
                    cors_list = cors[(prob_level, prob_type)]
                    print("{} Level {} Accuracy = {}/{} = {:.3f}".format(prob_type, prob_level, np.sum(cors_list), len(cors_list), np.mean(cors_list)))
                    f.write("{} Level {} Accuracy = {}/{} = {:.3f}\n".format(prob_type, prob_level, np.sum(cors_list), len(cors_list), np.mean(cors_list)))

        print("#####################")
        f.write("#####################\n")
        # also get accuracies for each
        for level in sorted(level_cors):
            cors_list = level_cors[level]
            print("Level {} Accuracy = {}/{} = {:.3f}".format(level, np.sum(cors_list), len(cors_list), np.mean(cors_list)))
            f.write("Level {} Accuracy = {}/{} = {:.3f}\n".format(level, np.sum(cors_list), len(cors_list), np.mean(cors_list)))
        print("#####################")
        f.write("#####################\n")

        for subject in subjects:
            # for subject in sorted(subject_cors):
            if subject in subject_cors:
                cors_list = subject_cors[subject]
                print("{} Accuracy = {}/{} = {:.3f}".format(subject, np.sum(cors_list), len(cors_list), np.mean(cors_list)))
                f.write("{} Accuracy = {}/{} = {:.3f}\n".format(subject, np.sum(cors_list), len(cors_list), np.mean(cors_list)))
        print("#####################")
        f.write("#####################\n")

        print("Overall Accuracy = {}/{} = {:.3f}".format(correct, total, correct/total))
        print("Skipped = {}".format(skipped))
        f.write("Overall Accuracy = {}/{} = {:.3f}\n".format(correct, total, correct/total))
        f.write("Skipped = {}".format(skipped))

    print()

def get_model_output(context, full_output, tokenizer):
    """
    Given the context and the full model output (context + generated),
    extract just the generated tokens.
    Remove the last token if it is <|endoftext|>
    """
    ret = full_output[len(context):]
    if ret[-1] == tokenizer.eos_token_id:
        ret = ret[:-1]
    return ret

def get_dataset(args):
    all_datasets = []

    if args.math_dataroot is not None:
        if args.math_mode == 'gpt2-eval':
            all_datasets.append(
                MATHDataset(
                    dataroot=args.math_dataroot,
                    tokenizer=None, # Set in run_training(), not in dataset creation
                    max_tokens=384 if args.arch == 'gpt2-xl' else 1024,
                    mode='gpt2-eval',
                )
            )
        else:
            all_datasets.append(
                MATHDataset(
                    dataroot=args.math_dataroot,
                    tokenizer=None, # Set in run_training(), not in dataset creation
                    max_tokens=384 if args.arch == 'gpt2-xl' else 1024,
                    mode='gpt2-eval',
                    mode_answer=args.math_mode,
                    peek_fraction=args.peek_fraction
                )
            )


    train_data = torch.utils.data.ConcatDataset(all_datasets)
    return train_data